In [2]:
# Python
import pandas as pd
import numpy as np
#
import xgboost
from xgboost import XGBRegressor
#
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [3]:
df = pd.read_csv('../../../data/df_monthly_returns_complete_percentage.csv', index_col='Date')

In [4]:
df

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
Date,,,,,,,,,,,,,,,,,,,,,
1999-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-11-01,1.13,1.00,1.00,1.00,1.03,1.10,1.01,1.03,1.00,1.03,...,1.00,1.00,1.00,0.93,1.00,0.99,1.00,1.00,1.00,0.84
1999-12-01,1.11,1.00,1.00,1.00,0.92,1.02,1.10,1.00,1.00,0.97,...,1.00,1.00,1.00,1.18,1.00,1.01,1.00,1.00,1.00,0.93
2000-01-01,1.02,1.00,1.00,1.00,0.96,0.88,0.89,0.73,1.00,1.00,...,1.00,1.00,1.00,0.85,1.00,1.22,1.00,1.00,1.00,0.93
2000-02-01,1.00,1.01,0.81,1.23,1.08,0.96,1.25,1.95,1.18,0.93,...,1.03,0.94,1.07,1.04,1.47,1.97,1.13,0.95,1.01,0.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-01,0.95,1.10,1.06,1.02,1.14,1.12,1.12,1.06,1.06,1.06,...,1.09,1.07,1.16,1.05,0.99,1.08,1.03,0.98,0.86,0.98
2024-06-01,1.00,0.96,0.96,0.97,0.95,0.96,0.94,0.94,0.95,1.11,...,1.07,0.91,1.18,0.89,1.01,0.90,1.19,1.01,1.14,0.98
2024-07-01,1.19,1.08,1.02,1.06,1.12,1.22,1.05,1.09,1.07,0.92,...,1.10,1.07,1.06,1.16,0.94,0.96,1.14,1.28,0.90,1.09


In [5]:
import torch

train_size = int(len(df) * 0.8)
test_size = len(df) - train_size
train, test = df[:train_size], df[train_size:]

In [10]:
train.T

Date,1999-10-01,1999-11-01,1999-12-01,2000-01-01,2000-02-01,2000-03-01,2000-04-01,2000-05-01,2000-06-01,2000-07-01,...,2018-12-01,2019-01-01,2019-02-01,2019-03-01,2019-04-01,2019-05-01,2019-06-01,2019-07-01,2019-08-01,2019-09-01
RS1.L,NaN,1.13,1.11,1.02,1.00,1.00,1.00,1.00,1.00,1.01,...,0.99,1.07,1.02,1.01,1.15,0.94,1.04,0.98,0.95,1.11
KE,NaN,1.00,1.00,1.00,1.01,1.00,1.00,1.00,1.00,1.00,...,0.88,1.04,0.96,1.00,0.98,0.94,1.14,0.98,0.83,1.10
TEG.DE,NaN,1.00,1.00,1.00,0.81,1.12,1.00,0.90,1.12,1.00,...,0.94,1.11,0.93,1.07,0.91,1.05,1.00,1.05,0.99,0.99
LEG.DE,NaN,1.00,1.00,1.00,1.23,1.03,1.00,1.00,1.00,1.00,...,0.90,1.12,0.96,1.12,0.95,1.03,0.96,1.06,1.02,0.98
SCS,NaN,1.03,0.92,0.96,1.08,0.98,0.99,1.04,1.39,1.01,...,0.92,1.12,1.06,0.83,1.20,0.93,1.07,1.00,0.92,1.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
KVHI,NaN,0.99,1.01,1.22,1.97,1.03,0.69,0.99,1.16,0.95,...,0.91,1.09,1.02,0.90,0.96,0.95,1.17,0.93,0.92,1.15
MOON.L,NaN,1.00,1.00,1.00,1.13,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,0.97,1.00,1.00,1.00,1.00,1.00,0.99,1.27
NEO,NaN,1.00,1.00,1.00,0.95,1.01,1.00,1.05,0.96,1.05,...,0.77,1.32,1.18,1.04,1.02,1.04,1.01,1.11,1.03,0.77
6055.T,NaN,1.00,1.00,1.00,1.01,1.46,1.00,0.69,1.90,0.53,...,0.81,1.17,1.12,0.96,1.23,0.83,1.12,0.93,0.76,1.22


In [11]:
# Exploding the list into separate rows
df_exploded = train.T.explode('values', ignore_index=True)


KeyError: 'values'

In [4]:
print("xgboost", xgboost.__version__)

xgboost 2.1.4


In [ ]:
df

In [ ]:
reg = XGBRegressor(n_estimators=1000)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        early_stopping_rounds=50,
        verbose=False)